In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
import seaborn as sns

In [ ]:
customers = pd.read_csv("Olist Brazilian E-Commerce Data/olist_customers_dataset.csv")
geolocations = pd.read_csv("Olist Brazilian E-Commerce Data/olist_geolocation_dataset.csv")
orderItems = pd.read_csv("Olist Brazilian E-Commerce Data/olist_order_items_dataset.csv")
orderPayments = pd.read_csv("Olist Brazilian E-Commerce Data/olist_order_payments_dataset.csv")
orderReviews = pd.read_csv("Olist Brazilian E-Commerce Data/olist_order_reviews_dataset.csv")
orders = pd.read_csv("Olist Brazilian E-Commerce Data/olist_orders_dataset.csv")
products = pd.read_csv("Olist Brazilian E-Commerce Data/olist_products_dataset.csv")
sellers = pd.read_csv("Olist Brazilian E-Commerce Data/olist_sellers_dataset.csv")
productCategoryName = pd.read_csv("Olist Brazilian E-Commerce Data/product_category_name_translation.csv")

In [ ]:
#orders['formatted_order_purchase_timestamp'] = pd.to_datetime(orders.order_purchase_timestamp,unit = 's')
orders['purchase_year'] = orders.order_purchase_timestamp.str.slice(0,4)
orders.purchase_year = orders.purchase_year.astype(int)
orders['purchase_month'] = orders.order_purchase_timestamp.str.slice(5,7)
orders.purchase_month = orders.purchase_month.astype(int)

monthly_transaction = orders.groupby(['purchase_year','purchase_month']).size()

orderItemsUnique = orderItems[orderItems.order_item_id == 1]

ordersWithItems = orders.merge(orderItemsUnique, on='order_id')[['order_id','customer_id','purchase_year','purchase_month','product_id','seller_id','price']]
monthly_amount = ordersWithItems.groupby(['purchase_year','purchase_month']).price.sum()

In [ ]:
#1. What is the growth of the company?

In [ ]:
#1-a. How many transactions were made each year by month?

In [ ]:
print(monthly_transaction)

In [ ]:
monthly_transaction.plot(kind='line', title = 'Transaction Counts')
sns.mpl.pyplot.ylabel('Counts')
sns.mpl.pyplot.xlabel('Year, Month')

In [ ]:
#1-b. How many dollar sales were generated each year by month?

In [ ]:
print(monthly_amount)

In [ ]:
#2. Who are the best sellers by amount? by number of transactions?

In [ ]:
bestSeller = ordersWithItems.groupby('seller_id')
print('best seller by amount:\n', bestSeller.size().sort_values(ascending=False).head(1))
print('best seller by number of transactions:\n', bestSeller.price.sum().sort_values(ascending=False).head(1))

In [ ]:
#3. What are the trends of top 5 best selling product type over the years?

In [ ]:
Category_Translation = products.merge(productCategoryName, on='product_category_name')[['product_id','product_category_name','product_category_name_english']]
ordersWithItemsCategory = ordersWithItems.merge(Category_Translation, on='product_id')
del ordersWithItemsCategory['product_category_name']
categorySales = ordersWithItemsCategory.groupby('product_category_name_english').price.sum().sort_values(ascending=False).head(5).index
categorySalesTop5 = ordersWithItemsCategory[ordersWithItemsCategory.product_category_name_english.isin(categorySales)]
categorySalesAmount = categorySalesTop5.groupby(['purchase_year','purchase_month','product_category_name_english']).sum()
unstackedCategorySalesAmount = categorySalesAmount.unstack()

In [ ]:
unstackedCategorySalesAmount.plot(title = 'Top 5 Best Selling Product Type',
                                  subplots=True, figsize=(15,4), layout=(1,5), sharey=True)

In [ ]:
#4. Is there any customer location that showed significant growth over the years?

In [ ]:
customers.head()
orders.head()
ordersWithCustomers = ordersWithItems.merge(customers, on='customer_id')[['order_id','customer_id','purchase_year','purchase_month','price','customer_zip_code_prefix','customer_city','customer_state']]
customerState = ordersWithCustomers.groupby(['purchase_year','customer_state']).price.sum()
unstackedCustomerState = customerState.unstack()

customerStateByYear = unstackedCustomerState.T
customerStateByYear['Growth'] = customerStateByYear[2018] - customerStateByYear[2016]
top5State = customerStateByYear.Growth.sort_values(ascending=False).head(5).index
print(top5State)

unstackedCustomerState.plot(kind='bar')

#customerState.sort_values(ascending=False).head()

In [ ]:
#5. How much sales can we expect to see in the rest of 2018 based of the sales growth from 2016 to 2017 